# Introduction of Stateful LSTM

In Recurrent Neural Networks, we are quickly confronted to the so-called gradient vanishing problem:

In machine learning, the vanishing gradient problem is a difficulty found in training artificial neural networks with gradient-based learning methods and backpropagation. In such methods, each of the neural network’s weights receives an update proportional to the gradient of the error function with respect to the current weight in each iteration of training. Traditional activation functions such as the hyperbolic tangent function have gradients in the range (−1,1)(−1,1) or [0,1)[0,1), and backpropagation computes gradients by the chain rule. This has the effect of multiplying n of these small numbers to compute gradients of the “front” layers in an n-layer network, meaning that the gradient (error signal) decreases exponentially with n and the front layers train very slowly.

One solution is to consider adding the updates instead of multiplying them, and this is exactly what the LSTM does. The state of every cell is updated in an additive way ($ c_t = ... $) such that the gradient hardly vanishes.

$ i_t = \sigma(W_{xi}x_t + W_{hi}h_{t-1}+W_{ci}c_{t-1} + b_i) $

$ f_t = \sigma(W_{xf}x_t + W_{hf}h_{t-w} + W_{cf}c_{t-1} + b_f) $

$ c_t = f_tc_{t-1} + i_ttanh(W_{xc}x_t+W_{hc}h_{t-1}+b_c) $

$ o_t = \sigma(W_{xo}x_t + W_{ho}h_{t-1} + W_{co}c_t + b_o) $

$ h_t = o_ttanh(c_t) $

### Questions and Answers

#### I’m given a big sequence (e.g. Time Series) and I split it into smaller sequences to construct my input matrix XX. Is it possible that the LSTM may find dependencies between the sequences?
No it’s not possible unless you go for the stateful LSTM. Most of the problems can be solved with stateless LSTM so if you go for the stateful mode, make sure you really need it. In stateless mode, long term memory does not mean that the LSTM will remember the content of the previous batches.

#### Why do we make the difference between stateless and stateful LSTM in Keras? 
A LSTM has cells and is therefore stateful by definition (not the same stateful meaning as used in Keras). Fabien Chollet gives this definition of statefulness: 
*stateful: Boolean (default False). If True, the last state for each sample at index i in a batch will be used as initial state for the sample of index i in the following batch. *

Said differently, whenever you train or test your LSTM, you first have to build your input matrix $X$ of shape **nb_samples, timesteps, input_dim** where your **batch size** divides **nb_samples**. For instance, if *nb_samples=1024* and *batch_size=64*, it means that your model will receive blocks of 64 samples, compute each output (whatever the number of timesteps is for every sample), average the gradients and propagate it to update the parameters vector. 

By default, Keras shuffles (permutes) the samples in $X$ and the dependencies between $X_i$ and $X_{i+1}$ are lost. Let’s assume there’s *no shuffling* in our explanation. 

If the model is stateless, the cell states are reset at each sequence. With the stateful model, all the states are propagated to the next batch. It means that the state of the sample located at index $i$, $X_i$ will be used in the computation of the sample $X_{i+bs}$ in the next batch, where $bs$ is the batch size (no shuffling).

#### Why do Keras require the batch size in stateful mode? 
When the model is stateless, Keras allocates an array for the states of size output_dim (understand number of cells in your LSTM). At each sequence processing, this state array is reset. 

In Stateful model, Keras must propagate the previous states for each sample across the batches. Referring to the explanation above, a sample at index $i$ in batch #1 $X_{i+bs}$ will know the states of the sample ii in batch #0 $(X_i)$. In this case, the structure to store the states is of the shape (batch_size, output_dim). This is the reason why you have to specify the batch size at the creation of the LSTM. If you don’t do so, Keras may raise an error to remind you: If a RNN is stateful, a complete input_shape must be provided (including batch size)

Code below gives intuitive example

In [1]:
T = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y', 'z','1','2','3','4','5','6']

SEQ_LEN = 4
BATCH_SIZE = 2
BATCH_CHARS  = int(len(T) / BATCH_SIZE)

x = [[0,0,0,0],[0,0,0,0]]

print("Sequence: ", '  '.join(str(c) for c in T))
for i in range(0, BATCH_CHARS - SEQ_LEN +1, SEQ_LEN):
    print('BATCH', i/SEQ_LEN)
    for batch_idx in range(BATCH_SIZE):
        start = batch_idx * BATCH_CHARS + i
        print('\tsequence', batch_idx, 'of batch:',)
        for j in range(SEQ_LEN):
            x[batch_idx][j] = T[start+j]
            print(T[start+j],)
    #here we would yield x (the batch) if this were an iterator
        print()

Sequence:  a  b  c  d  e  f  g  h  i  j  k  l  m  n  o  p  q  r  s  t  u  v  w  x  y  z  1  2  3  4  5  6
BATCH 0.0
	sequence 0 of batch:
a
b
c
d

	sequence 1 of batch:
q
r
s
t

BATCH 1.0
	sequence 0 of batch:
e
f
g
h

	sequence 1 of batch:
u
v
w
x

BATCH 2.0
	sequence 0 of batch:
i
j
k
l

	sequence 1 of batch:
y
z
1
2

BATCH 3.0
	sequence 0 of batch:
m
n
o
p

	sequence 1 of batch:
3
4
5
6



In [7]:
import numpy 
t = numpy.reshape(T, (2,4,4))

In [8]:
print(t[0][0])
print(t[1][0])

['a' 'b' 'c' 'd']
['q' 'r' 's' 't']


The logic goes roughly like this:

    BatchSize = 5
    sequences = [ X[I::BatchSize] for I in xrange(BatchSize) ]
    batches   = [ sequences[:, N] for N in xrange(X.shape[0]/BatchSize) ]
    sequences[0] == [0, 5, 10, …]
    sequences[1] == [1, 6, 11, …]
    sequences[2] == [2, 7, 12, …]
    sequences[3] == [3, 8, 13, …]
    sequences[4] == [4, 9, 14, …]

    batches[0] == [ 0,  1,  2,  3,  4]
    batches[1] == [ 5,  6,  7,  8,  9]
    batches[2] == [10, 11, 12, 13, 14]
